In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import numpy as np

import researchpy as rp

In [2]:
import statsmodels.datasets
systolic = statsmodels.datasets.webuse('systolic')

systolic.iloc[0, 0] = np.nan

In [3]:
#from pystata import Stata
import stata_setup

stata_setup.config("C:/Program Files/Stata17/", "be")


  ___  ____  ____  ____  ____ ®
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       BE—Basic Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user  perpetual
Serial number: 301706367794
  Licensed to: Corey Bryant
               University of Michigan

Notes:
      1. Unicode is supported; see help unicode_advice.


## Descriptive Information

In [4]:
%%stata -d systolic

codebook



. 
. codebook

-------------------------------------------------------------------------------
drug                                                                (unlabeled)
-------------------------------------------------------------------------------

                  Type: Numeric (double)

                 Range: [1,4]                         Units: 1
         Unique values: 4                         Missing .: 1/58

            Tabulation: Freq.  Value
                           14  1
                           15  2
                           12  3
                           16  4
                            1  .

-------------------------------------------------------------------------------
disease                                                             (unlabeled)
-------------------------------------------------------------------------------

                  Type: Numeric (int)

                 Range: [1,3]                         Units: 1
         Unique values: 3

In [5]:
rp.codebook(systolic)

Variable: drug    Data Type: float64 

 Number of Obs.: 58 
 Number of missing obs.: 1 
 Percent missing: 1.72 
 Number of unique values: 4 

 Range: [1.0, 4.0] 
 Mean: 2.53 
 Standard Deviation: 1.15 
 Mode: 4.0 
 10th Percentile: 1.0 
 25th Percentile: 2.0 
 50th Percentile: 2.0 
 75th Percentile: 4.0 
 90th Percentile: 4.0 





Variable: disease    Data Type: int16 

 Number of Obs.: 58 
 Number of missing obs.: 0 
 Percent missing: 0.0 
 Number of unique values: 3 

 Range: [1, 3] 
 Mean: 2.02 
 Standard Deviation: 0.83 
 Mode: 3 
 10th Percentile: 1.0 
 25th Percentile: 1.0 
 50th Percentile: 2.0 
 75th Percentile: 3.0 
 90th Percentile: 3.0 





Variable: systolic    Data Type: int16 

 Number of Obs.: 58 
 Number of missing obs.: 0 
 Percent missing: 0.0 
 Number of unique values: 32 

 Range: [-6, 44] 
 Mean: 18.88 
 Standard Deviation: 12.8 
 Mode: 1 
 10th Percentile: 1.0 
 25th Percentile: 9.0 
 50th Percentile: 21.0 
 75th Percentile: 28.0 
 90th Percentile: 34.0 







In [ ]:
%%stata 

tab drug

In [ ]:
rp.summary_cat(systolic.drug)

In [ ]:
%%stata

tab drug disease, chi V

In [ ]:
tab, res = rp.crosstab(systolic.drug, systolic.disease, test = "chi-square")

print(tab, res, sep = "\n"*2)

### Mean

Stata considers each group a sub-population when using **-mean *varlist*, over(*var*)-**; in doinging so, a different formula is used.

In [ ]:
%%stata

tab drug, summarize(systolic)

mean systolic, over(drug)

In [ ]:
rp.summarize(systolic.groupby("drug")["systolic"])

## T-tests

### Independent Sample's T-Test (equal variance)

In [ ]:
%%stata

ttest systolic if drug == 1 | drug == 2, by(drug)

esize twosample systolic if drug == 1 | drug == 2, by(drug) all

In [ ]:
des, res = rp.difference_test("systolic ~ C(drug)", data = systolic.query(f"drug in {[1, 2]}")).conduct(effect_size = "all")

print(des, res, sep = "\n"*2)

In [ ]:
des, res = rp.ttest(systolic.query("drug == 1")["systolic"], 
                    systolic.query("drug == 2")["systolic"],
                    group1_name = "1", group2_name = "2")

print(des, res, sep = "\n"*2)

### Independent Sample's T-Test (unequal variance)

#### Satterthwaite Degrees of Freedom

In [ ]:
%%stata

ttest systolic if drug == 1 | drug == 2, by(drug) unequal

esize twosample systolic if drug == 1 | drug == 2, by(drug) unequal all

In [ ]:
des, res = rp.difference_test("systolic ~ C(drug)", data = systolic.query(f"drug in {[1, 2]}"), equal_variances = False).conduct(effect_size = "all")

print(des, res, sep = "\n"*2)

In [ ]:
des, res = rp.ttest(systolic.query("drug == 1")["systolic"], 
                    systolic.query("drug == 2")["systolic"],
                    group1_name = "1", group2_name = "2",
                    equal_variances = False)

print(des, res, sep = "\n"*2)

#### Welch's Degrees of Freedom

In [ ]:
%%stata

ttest systolic if drug == 1 | drug == 2, by(drug) welch

esize twosample systolic if drug == 1 | drug == 2, by(drug) welch all

In [ ]:
des, res = rp.difference_test("systolic ~ C(drug)", data = systolic.query(f"drug in {[1, 2]}"), equal_variances = False, welch_dof = "welch").conduct(effect_size = "all")

print(des, res, sep = "\n"*2)

In [ ]:
des, res = rp.ttest(systolic.query("drug == 1")["systolic"], 
                    systolic.query("drug == 2")["systolic"],
                    group1_name = "1", group2_name = "2",
                    equal_variances = False,
                    welch_dof = "welch")

print(des, res, sep = "\n"*2)

## ANOVA

###

In [7]:
%%stata

anova systolic i.drug##i.disease

estat esize

estat esize, epsilon

estat esize, omega


. 
. anova systolic i.drug##i.disease

                         Number of obs =         57    R-squared     =  0.4443
                         Root MSE      =    10.4225    Adj R-squared =  0.3084

                  Source | Partial SS         df         MS        F    Prob>F
            -------------+----------------------------------------------------
                   Model |  3907.9272         11   355.26611      3.27  0.0024
                         |
                    drug |  2813.6905          3   937.89683      8.63  0.0001
                 disease |  349.82654          2   174.91327      1.61  0.2111
            drug#disease |  706.68708          6   117.78118      1.08  0.3863
                         |
                Residual |  4888.2833         45   108.62852  
            -------------+----------------------------------------------------
                   Total |  8796.2105         56   157.07519  

. 
. estat esize

Effect sizes for linear models

-----------------

In [6]:
## Epsilon and Omega partials doesn't match up with Stata
rp.anova("systolic ~ C(drug)*C(disease)", data = systolic, sum_of_squares = 3).results()



 Note: Effect size values for factors are partial. 




(                        0
 Number of obs =   57.0000
 Root MSE =        10.4225
 R-squared =        0.4443
 Adj R-squared =    0.3084,
          Source Sum of Squares Degrees of Freedom Mean Squares  F value p-value Eta squared Epsilon squared Omega squared
 0         Model      3907.9272                 11     355.2661   3.2705  0.0024      0.4443          0.3084        0.3047
 1                                                                                                                        
 2          drug      1046.6286                3.0     348.8762   3.2116  0.0317      0.1764          0.0732        0.1043
 3       disease      3573.6857                2.0    1786.8428  16.4491     0.0      0.4223          0.2713        0.3515
 4  drug:disease       3179.816                6.0     529.9693   4.8787  0.0006      0.3941          0.2111        0.2899
 5                                                                                                                        
 6 

In [ ]:
# Stata 17 calculates Omega2 based on the F value

(9.046 - 1) / (9.046 + (46 + 1) / 3)

In [ ]:
# Stata 13 calculates Omega2 based on the replationship to Eta2

0.3711 - (3/46) * (1 - 0.3711)

In [ ]:
(11 * (387.2126 - 110.4525)) / (9340.1552 + 110.4525)

In [ ]:
(3 * (999.1573 - 110.4525)) / (3 * 999.1573 + ((58 - 3) * 110.4525))

In [ ]:
%%stata

anova systolic i.drug i.disease

estat esize

estat esize, epsilon

estat esize, omega

In [ ]:
rp.anova("systolic ~ C(drug) + C(disease)", data = systolic, sum_of_squares = 3).results()

In [ ]:
systolic.to_clipboard()

In [ ]:
import patsy

In [ ]:
dv, iv = patsy.dmatrices(f"systolic ~ C(drug) + C(disease)", systolic, 1)

In [ ]:
iv

In [ ]:
np.asarray(iv)[:10, :]

In [ ]:
systolic.disease.unique()

In [ ]:
design_info2 = iv.design_info.subset('C(disease)')

design_info2

In [ ]:
design_info2.slice('C(disease)')

In [ ]:
iv.design_info.term_slices

In [ ]:
# patsy.build_design_matrices([design_info2], iv)[0]

In [ ]:
iv.design_info.term_name_slices.items()

In [ ]:
for key, val in iv.design_info.term_name_slices.items():
    print(key, val)
    print(iv.design_info.slice(val))
    
    print(iv[iv.design_info.slice(val)])
    
    print("\n"*3)
    

In [ ]:
dis = iv[iv.design_info.slice('C(disease)')]
dis

In [ ]:
dis @ iv.T

In [ ]:
iv.design_info.column_name_indexes['C(disease)']

In [ ]:
iv[iv.design_info.slice('C(disease)')]

Trying based on subset

In [ ]:
import patsy
dv, iv = patsy.dmatrices(f"systolic ~ C(drug)*C(disease)", systolic, 1)

iv.design_info

In [ ]:
disease_info = iv.design_info.subset('C(disease)')
disease_info

In [ ]:
drug_info = iv.design_info.subset('C(drug)')
drug_info

In [ ]:
diseasedrug_info = iv.design_info.subset('C(drug):C(disease)')
diseasedrug_info

In [ ]:
iv.design_info.term_names[1:]

In [ ]:
#patsy.design_matrix_builders(iv.design_info.term_names[1:], next(iter(iv.design_info.term_names[1:])), eval_env = 1)

#terms_design_info = {}
#for term in iv.design_info.term_names[1:]:
    #terms_design_info[term] =  iv.design_info.subset(term)

terms_design_info = []
for term in iv.design_info.term_names[1:]:
    terms_design_info.append(iv.design_info.subset(term))
    
terms_design_info


In [ ]:
iv.design_info.term_names

In [ ]:
np.linalg.matrix_rank(factor_matrices['C(drug)'] - 1)

In [ ]:
factor_matrices = {}
for factor in patsy.build_design_matrices(terms_design_info, systolic):
    factor_matrices[factor.design_info.term_names[1]] = np.asarray(factor)
    
factor_matrices
    

In [ ]:
patsy.build_design_matrices([disease_info, drug_info], systolic)[1]

In [ ]:
#np.asarray(patsy.build_design_matrices([disease_info, iv.design_info.subset('C(disease)')], np.asarray(iv)))[0]